In [2]:
from ipynb.fs.full.boundaryCellDisplacement import *

In [3]:
class boundaryCellTraction(boundaryCellDisplacement, A):

    def __init__(self, edges, xy):
        
        sw_S, sw_N, sw_W, sw_E = 1, 1, 1, 1

        for edge in edges:
            if edge == "b": sw_S = 0
            if edge == "t": sw_N = 0
            if edge == "l": sw_W = 0
            if edge == "r": sw_E = 0
        
        if transient:
            self.a_P = (rho*dx*dy/(dt**2)) + A(xy).a_N*sw_N + A(xy).a_S**sw_S + A(xy).a_E*sw_E + A(xy).a_W*sw_W
        else:
            self.a_P = A(xy).a_N*sw_N + A(xy).a_S**sw_S + A(xy).a_E*sw_E + A(xy).a_W*sw_W

        self.a_N = A(xy).a_N
        self.a_S = A(xy).a_S 
        self.a_E = A(xy).a_E 
        self.a_W = A(xy).a_W 

        # self.a_faces = A(xy).a_N*sw_N + A(xy).a_S*sw_S + A(xy).a_E*sw_E + A(xy).a_W*sw_W 

    def b_temp(u_old, u_old_old, k, xy):      
        return A.b_temp(u_old, u_old_old, k, xy)
    
    def b_diff(edges, k, xy, u_previous):

        if xy == "x":
            uv = "v"
        if xy == "y":
            uv = "u"
            
        N_term =(
                    Sfy*A.coef(xy, "N", uv)*(
                    (boundaryCellDisplacement.corner(edges, "NE", uv, u_previous, k) - boundaryCellDisplacement.corner(edges, "NW", uv, u_previous, k))
                    /dx)
                )
        S_term =(
                    Sfy*A.coef(xy, "S", uv)*(
                        (boundaryCellDisplacement.corner(edges, "SE", uv, u_previous, k) - boundaryCellDisplacement.corner(edges, "SW", uv, u_previous, k))
                        /dx)
                ) 
        E_term =(
                    Sfx*A.coef(xy, "E", uv)*(
                        (boundaryCellDisplacement.corner(edges, "NE", uv, u_previous, k) - boundaryCellDisplacement.corner(edges, "SE", uv, u_previous, k))
                        /dy)
                ) 
        W_term =(
                    Sfx*A.coef(xy, "W", uv)*(
                        (boundaryCellDisplacement.corner(edges, "NW", uv, u_previous, k) - boundaryCellDisplacement.corner(edges, "SW", uv, u_previous, k))
                        /dy)
                )

        for edge in edges:

            if (edge == "b") & (xy == "x") : S_term =  Sfy*tr_bottom_x  
            if (edge == "b") & (xy == "y") : S_term =  Sfy*tr_bottom_y  
            if (edge == "t") & (xy == "x") : N_term =  Sfy*tr_top_x  
            if (edge == "t") & (xy == "y") : N_term =  Sfy*tr_top_y  
            if (edge == "l") & (xy == "x") : W_term =  Sfx*tr_left_x  
            if (edge == "l") & (xy == "y") : W_term =  Sfx*tr_left_y  
            if (edge == "r") & (xy == "x") : E_term =  Sfx*tr_right_x  
            if (edge == "r") & (xy == "y") : E_term =  Sfx*tr_right_y        

        b_diffusion = (N_term + S_term + E_term + W_term)

        return b_diffusion
    
    # def corner(edges, corner_placement, uv, u_previous, k):

    #     if uv == "u":
    #         uv_i = 0
    #     elif uv == "v":
    #         uv_i = 1

    #     disp = displacement(k, u_previous, uv_i)

    #     if corner_placement == "NE":
    #         corner =  (1/4)*(disp.P + disp.NE + disp.N + disp.E)
    #     if corner_placement == "SE":
    #         corner =  (1/4)*(disp.P + disp.SE + disp.S + disp.E)
    #     if corner_placement == "SW":
    #         corner =  (1/4)*(disp.P + disp.SW + disp.S + disp.W)
    #     if corner_placement == "NW":
    #         corner =  (1/4)*(disp.P + disp.NW + disp.N + disp.W)

    #     for edge in edges:
    #         if (edge == "b") & (corner_placement == "SE"):
    #             corner =  (1/2)*(disp.SE + disp.S)
    #         if (edge == "b") & (corner_placement == "SW"):
    #             corner =  (1/2)*(disp.SW + disp.S)

    #         if (edge == "t") & (corner_placement == "NE"):
    #             corner =  (1/2)*(disp.NE + disp.N)
    #         if (edge == "t") & (corner_placement == "NW"):
    #             corner =  (1/2)*(disp.NW + disp.N)

    #         if (edge == "l") & (corner_placement == "NW"):
    #             corner =  (1/2)*(disp.NW + disp.W)
    #         if (edge == "l") & (corner_placement == "SW"):
    #             corner =  (1/2)*(disp.SW + disp.W)

    #         if (edge == "r") & (corner_placement == "NE"):
    #             corner =  (1/2)*(disp.NE + disp.E)
    #         if (edge == "r") & (corner_placement == "SE"):
    #             corner =  (1/2)*(disp.SE + disp.E)
        
    #     return corner

# boundaryCellTraction.b_diff(["b", "l"], 14, "x", u_previous)
   